<a href="https://colab.research.google.com/github/jeein2222/gradProject/blob/lyrics/generation_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive',force_remount=True)

In [ ]:
!pip install ratsnlp

In [ ]:
import torch
from ratsnlp.nlpbook.generation import GenerationTrainArguments
args=GenerationTrainArguments(
    pretrained_model_name="skt/kogpt2-base-v2",
    downstream_corpus_name="lyrics",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-generation2",
    max_seq_length=32,
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5, #1회 스텝에서 모델을 얼마나 업데이트할지에 관한 크기
    epochs=3, #학습을 몇 번 반복
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7 #랜덤 시드
)

In [ ]:
from ratsnlp import nlpbook
nlpbook.set_seed(args)

nlpbook.set_logger(args)

In [ ]:
from transformers import PreTrainedTokenizerFast
tokenizer=PreTrainedTokenizerFast.from_pretrained(
    args.pretrained_model_name,
    eos_token="</s>"
)

In [ ]:
import os
from ratsnlp.nlpbook.generation.corpus import GenerationExample
class LyricsCorpus:
    def __init__(self):
        pass
    
    def get_examples(self,data_root_path,mode):
        data_fpath=os.path.join(data_root_path,f"{mode}.txt")
        lines=open(data_fpath,"r",encoding="utf-8").readlines()
        examples=[]
        for (i,line) in enumerate(lines):
           if i==0:
               continue
           examples.append(GenerationExample(text=line))
        return examples



In [ ]:
import os
os.chdir('/gdrive/MyDrive/Colab Notebooks')
!pwd

In [ ]:
from ratsnlp.nlpbook.generation import GenerationDataset

corpus = LyricsCorpus()
train_dataset = GenerationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)